In [67]:
import pandas as pd
import sklearn
import numpy as np
import scipy 
from os import listdir
from os.path import isfile, join
import json
import glob
import nltk
import gensim
from gensim.models.phrases import Phrases, Phraser
from sklearn.model_selection import train_test_split


In [68]:
mypath = "/Users/admin/Desktop/phd/CS-685-project/"
dataDir = mypath+"/data/"
allFiles = [f for f in listdir(dataDir) if isfile(join(dataDir, f))]
allFiles = [f for f in allFiles if "json" in f]

# allFiles = ['complaints_2020-10-08_fixed.json']

In [69]:
print(allFiles)
reports = []

for fi in allFiles:
    print(fi)
    for file in glob.glob(dataDir+fi):
        for line in open(file, "r"):
            #print(line[179])
#             print(line)
            reports.append(json.loads(line))
        

['complaints_Delhi.json', 'complaints_Odisha.json', 'complaints_Bangalore.json', 'complaints_2020-10-08_fixed_bangalore.json', 'complaints_Mumbai.json']
complaints_Delhi.json
complaints_Odisha.json
complaints_Bangalore.json
complaints_2020-10-08_fixed_bangalore.json
complaints_Mumbai.json


In [72]:

import pandas as pd
df = pd.DataFrame(reports)
df

,id,description,location,image_url,postedOn,category,subCategory,myCategory,page
0,1943280,"Z6-W61,SHIVA HOMES,A-sahu mangalam bhawan ke b...","E-111, Om Vihar Extn, Param Puri, Uttam Nagar,...",[],2020-12-02 T12:06:57,Animal Catcher,Dog,Dog,1
1,1922159,Sanitization is not here,"Sanjay Amar Colony, Vishwas Nagar, Shahdara, D...",[],2020-11-29 T13:06:58,Toilet My Right,No Water Supply In Public Toilet(s),No Water Supply In Public Toilet(s),1
2,1852336,"Z1-W41,Ram nagar nahar Kinare M. 9479010167","Shop No.AB-399, Amar Puri, Nabi karim, Paharga...",[],2020-11-18 T13:06:58,Solid Waste Management,Collection And Removal Of Garbage,Collection And Removal Of Garbage,1
3,1790436,"Z1,w60,AKANSHA KUNJ,A-Shiv mandir pas,h.no. 21...","B5 Rd, Pocket 14, Sector 9, Rohini, Delhi, 110085",[],2020-11-02 T13:06:58,Others,Other Services,Other Services,1
4,1788056,Z3 w34 aanand nagar A-fokat para sai mandir ke...,Raja tala,[],2020-11-02 T13:06:58,Solid Waste Management,Cleaning Of Drain,Cleaning Of Drain,1
...,...,...,...,...,...,...,...,...,...
46003,451071,"कांजूरमार्ग, भांडूप और मुलुंड मे जो मेट्रो का ...","Kanjurmarg West, Bhandup West, Mumbai, Maharas...",[],2019-12-03 T13:00:35,"Mobility - Roads, Footpaths and Infrastructure",Construction of new footpaths,Construction of new footpaths,14
46004,448830,"Despite being a MHADA building, there is no wa...","Marathon Mhada, Dainik Shivneri Road, Ganapatr...",[],2019-12-03 T13:00:35,Garbage and Unsanitary Practices,Implementation Of Waste Segregation/Garbage,Implementation Of Waste Segregation/Garbage,14
46005,448707,Broken Footpath full of debris near hiranandan...,"10-A, Orchard Ave, Hiranandani Gardens, Powai,...",[https://jccd-ichangemycity-complaints.s3.ap-s...,2019-12-03 T13:00:39,"Mobility - Roads, Footpaths and Infrastructure",Construction of new footpaths,Construction of new footpaths,14
46006,448255,Near by two tanks police station there is nawa...,"130, Ms Ali Road, Ajmer, Kumbharwada, Mumbai, ...",[https://jccd-ichangemycity-complaints.s3.ap-s...,2019-12-03 T13:00:40,Animal Husbandry,Stray Dog Sterilisation/Animal Birth Control (...,Stray Dog Sterilisation/Animal Birth Control (...,14


In [73]:
wordsList = [nltk.word_tokenize(text) for text in df["description"]]
cleanWordsList = []

def checkWord(word):
    if(len(word)<=1):
        return False
    for ch in word:
        if((ch>='a' and ch<= 'z') or (ch>='A' and ch<='Z')):
            return True
    return False
    
for words in wordsList:
    wordList = []
    for word in words:
        if(checkWord(word)):
            wordList.append(word.lower())
    cleanWordsList.append(wordList)
    
    
df["words"] = cleanWordsList 


In [74]:
set(df["category"])

{'Animal Catcher',
 'Animal Husbandry',
 'Certificates',
 'Community Infrastructure',
 'Covid 19',
 'Crime and Safety',
 'EMP Grievance Redressal',
 'Electricity & Power',
 'Electricity and Power Supply',
 'Fire Pollution',
 'Fire Safety',
 'Garbage and Unsanitary Practices',
 'Lakes',
 'Mobility - Roads, Footpaths and Infrastructure',
 'Mobility - Roads, Public transport',
 'Others',
 'PWD',
 'Parks & Recreation',
 'Pension',
 'Playgrounds',
 'Pollution',
 'Power supply',
 'Proibition & Sale of Tobacco products, Plastic carry bags',
 'Public Toilets',
 'Public Transport - BMTC',
 'Public Transport - KSRTC',
 'Ration Card',
 'Safety and Crime',
 'Sanitation',
 'Sewerage Systems',
 'Solid Waste Management',
 'Storm Water Drains',
 'Street lighting',
 'Toilet My Right',
 'Traffic and Road Safety',
 'Trees and Saplings',
 'Water Supply',
 'Water Supply Department',
 'Water Supply and Services'}

In [75]:
broadCat = {'Animal Catcher': 'Animal',
 'Animal Husbandry': 'Animal',
 'Certificates':'other',
 'Community Infrastructure':'Infrastructure',
 'Covid 19':'covid',
 'Crime and Safety':'Crime and Safety',
 'EMP Grievance Redressal':'Grievance Redressal',
 'Electricity & Power':"Electricity",
 'Electricity and Power Supply':"Electricity",
 'Fire Pollution':"pollution",
 'Fire Safety':"Infrastructure",
 'Garbage and Unsanitary Practices':"cleanliness",
 'Lakes':"pollution",
 'Mobility - Roads, Footpaths and Infrastructure':"transport",
 'Mobility - Roads, Public transport':"transport",
 'Others':"other",
 'PWD':"Infrastructure",
 'Parks & Recreation':"lifestyle",
 'Pension':"Grievance Redressal",
 'Playgrounds':"lifestyle",
 'Pollution':"pollution",
 'Power supply':"Electricity",
 'Proibition & Sale of Tobacco products, Plastic carry bags':"pollution",
 'Public Toilets':"cleanliness",
 'Public Transport - BMTC':"transport",
 'Public Transport - KSRTC':"transport",
 'Ration Card':"Grievance Redressal",
 'Safety and Crime':"Crime and Safety",
 'Sanitation':"cleanliness",
 'Sewerage Systems':"Infrastructure",
 'Solid Waste Management':"pollution",
 'Storm Water Drains':"Infrastructure",
 'Street lighting':"Infrastructure",
 'Toilet My Right':"cleanliness",
 'Traffic and Road Safety':"transport",
 'Trees and Saplings':"Tree",
 'Water Supply':"water",
 'Water Supply Department':"water",
 'Water Supply and Services':"water"}

In [80]:
df["broadCategory"] = df["category"].apply(lambda x: broadCat[x])
print(set(df["broadCategory"]))
def split(df, col):
    trainDfs = []
    testDfs = []
    for cat in set(df[col]):
        tempDf = df[df[col] == cat]
        if(len(tempDf.index)<=2):
            tempTrain = tempDf
            tempTest = tempDf
        else:
            tempTrain, tempTest = train_test_split(tempDf, test_size=0.3, random_state=42)
        trainDfs.append(tempTrain)
        testDfs.append(tempTest)
    train = pd.concat(trainDfs)
    test = pd.concat(testDfs)
    return train,test

train,test = split(df, "broadCategory")
        
        

{'water', 'Grievance Redressal', 'Crime and Safety', 'transport', 'lifestyle', 'covid', 'Tree', 'Electricity', 'other', 'cleanliness', 'Infrastructure', 'Animal', 'pollution'}


In [81]:
from gensim.models.doc2vec import TaggedDocument
train_tagged = train.apply(
    lambda r: TaggedDocument(words=r["words"], tags=[r.broadCategory]), axis=1)
test_tagged = test.apply(
    lambda r: TaggedDocument(words=r['words'], tags=[r.broadCategory]), axis=1)

In [82]:
import multiprocessing
from tqdm import tqdm
from gensim.models import Doc2Vec
from sklearn import utils

cores = multiprocessing.cpu_count()

model_dbow = Doc2Vec(dm=0, vector_size=300, negative=5, hs=0, min_count=2, sample = 0, workers=cores)
model_dbow.build_vocab([x for x in tqdm(train_tagged.values)])

for epoch in range(30):
    model_dbow.train(utils.shuffle([x for x in tqdm(train_tagged.values)]), total_examples=len(train_tagged.values), epochs=1)
    model_dbow.alpha -= 0.002
    model_dbow.min_alpha = model_dbow.alpha
    
    

100%|██████████| 32199/32199 [00:00<00:00, 2957848.28it/s]


In [83]:
def vec_for_learning(model, tagged_docs):
    sents = tagged_docs.values
    targets, regressors = zip(*[(doc.tags[0], model.infer_vector(doc.words, steps=20)) for doc in sents])
    return targets, regressors

In [84]:

from sklearn.linear_model import LogisticRegression


y_train, X_train = vec_for_learning(model_dbow, train_tagged)
y_test, X_test = vec_for_learning(model_dbow, test_tagged)
logreg = LogisticRegression(n_jobs=1, C=1e5)
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)
from sklearn.metrics import accuracy_score, f1_score
print('Testing accuracy %s' % accuracy_score(y_test, y_pred))
print('Testing F1 score: {}'.format(f1_score(y_test, y_pred, average='weighted')))

Testing accuracy 0.4753421681512057
Testing F1 score: 0.44370347741769256


/Library/Python/3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [85]:
model_dmm = Doc2Vec(dm=1, dm_mean=1, vector_size=300, window=10, negative=5, min_count=1, workers=5, alpha=0.065, min_alpha=0.065)
model_dmm.build_vocab([x for x in tqdm(train_tagged.values)])

for epoch in range(30):
    model_dmm.train(utils.shuffle([x for x in tqdm(train_tagged.values)]), total_examples=len(train_tagged.values), epochs=1)
    model_dmm.alpha -= 0.002
    model_dmm.min_alpha = model_dmm.alpha

100%|██████████| 32199/32199 [00:00<00:00, 3164524.09it/s]


In [86]:
y_train, X_train = vec_for_learning(model_dmm, train_tagged)
y_test, X_test = vec_for_learning(model_dmm, test_tagged)
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)
print('Testing accuracy %s' % accuracy_score(y_test, y_pred))
print('Testing F1 score: {}'.format(f1_score(y_test, y_pred, average='weighted')))

Testing accuracy 0.5594901875588384
Testing F1 score: 0.5581157884412768


/Library/Python/3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [87]:
model_dbow.delete_temporary_training_data(keep_doctags_vectors=True, keep_inference=True)
model_dmm.delete_temporary_training_data(keep_doctags_vectors=True, keep_inference=True)

In [88]:
from gensim.test.test_doc2vec import ConcatenatedDoc2Vec
new_model = ConcatenatedDoc2Vec([model_dbow, model_dmm])

In [89]:
def get_vectors(model, tagged_docs):
    sents = tagged_docs.values
    targets, regressors = zip(*[(doc.tags[0], model.infer_vector(doc.words, steps=20)) for doc in sents])
    return targets, regressors

In [90]:
from sklearn.metrics import classification_report
y_train, X_train = get_vectors(new_model, train_tagged)
train = pd.DataFrame()
y_test, X_test = get_vectors(new_model, test_tagged)
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)
print('Testing accuracy %s' % accuracy_score(y_test, y_pred))
print('Testing F1 score: {}'.format(f1_score(y_test, y_pred, average='weighted')))

print(classification_report(y_test,y_pred))

/Library/Python/3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/Library/Python/3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Testing accuracy 0.5715113331885002
Testing F1 score: 0.5523233390055717
                     precision    recall  f1-score   support

             Animal       0.41      0.22      0.28       915
   Crime and Safety       0.00      0.00      0.00        35
        Electricity       0.50      0.01      0.03        69
Grievance Redressal       0.00      0.00      0.00         1
     Infrastructure       0.63      0.65      0.64      3100
               Tree       0.71      0.09      0.15       138
        cleanliness       0.55      0.59      0.57      3930
              covid       0.00      0.00      0.00         4
          lifestyle       0.33      0.02      0.04        51
              other       0.00      0.00      0.00       161
          pollution       1.00      0.01      0.01       186
          transport       0.57      0.65      0.61      5123
              water       0.86      0.06      0.12        96

           accuracy                           0.57     13809
          

In [91]:
from xgboost import XGBClassifier
model = XGBClassifier()
# print(X_train)

# print(list(y_train))
# train = pd.DataFrame()
# test = pd.DataFrame()
# train["feature"] = list(X_train)
# print(list(X_train)[0])
# train["label"] = list(y_train)
model.fit(np.array(X_train), np.array(y_train))
y_pred = model.predict(np.array(X_test))
print('Testing accuracy %s' % accuracy_score(y_test, y_pred))
print('Testing F1 score: {}'.format(f1_score(y_test, y_pred, average='weighted')))

Testing accuracy 0.5943949598088203
Testing F1 score: 0.5803036830871672


In [65]:
print(classification_report(y_test,y_pred))

                                                           precision    recall  f1-score   support

                                           Animal Catcher       0.00      0.00      0.00         4
                                         Animal Husbandry       0.50      0.21      0.30       912
                                             Certificates       0.43      0.37      0.40        84
                                 Community Infrastructure       0.41      0.05      0.10       353
                                                 Covid 19       0.00      0.00      0.00         4
                                         Crime and Safety       1.00      0.03      0.06        33
                                  EMP Grievance Redressal       0.00      0.00      0.00         1
                                      Electricity & Power       0.00      0.00      0.00         1
                             Electricity and Power Supply       0.37      0.41      0.39        68
         

In [66]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print('Testing accuracy %s' % accuracy_score(y_test, y_pred))
print('Testing F1 score: {}'.format(f1_score(y_test, y_pred, average='weighted')))
print(classification_report(y_test,y_pred))

Testing accuracy 0.3882940240196788
Testing F1 score: 0.2459410106272814


/Library/Python/3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                                                           precision    recall  f1-score   support

                                           Animal Catcher       0.00      0.00      0.00         4
                                         Animal Husbandry       0.00      0.00      0.00       912
                                             Certificates       0.00      0.00      0.00        84
                                 Community Infrastructure       0.00      0.00      0.00       353
                                                 Covid 19       0.00      0.00      0.00         4
                                         Crime and Safety       0.00      0.00      0.00        33
                                  EMP Grievance Redressal       0.00      0.00      0.00         1
                                      Electricity & Power       0.00      0.00      0.00         1
                             Electricity and Power Supply       0.00      0.00      0.00        68
         

In [ ]:
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
kernel = 1.0 * RBF(1.0)
gpc = GaussianProcessClassifier(kernel=kernel,random_state=0)
gpc.fit(X_train, y_train)
y_pred = gpc.predict(X_test)
print('Testing accuracy %s' % accuracy_score(y_test, y_pred))
print('Testing F1 score: {}'.format(f1_score(y_test, y_pred, average='weighted')))
print(classification_report(y_test,y_pred))